In [1]:
# 20 books per 13 genres per year --> years back to 2011
# for one year: 280 book covers
# for five years: 1400 book covers

#links on links on links

#fiction, historical fiction, mystery and thriller, romance, ROMANTASY (AXED), fantasy, science fiction, horror,
#young adult fantasy and science fiction, young adult fiction, nonfiction, memoir and autobiography, history and autobiography humor

#NEW LIST: fiction, historical fiction

##FUTURE REFERENCE: TRICKY ANALYSIS DECISION: NOT ALL THE GENRES WE INITIALLY CHOSE WERE FOUND IN ALL THE YEARS- WE HAD TO GO BACK AND
#ALTER OUR GENRE LIST TO REMAIN CONSISTENT THROUGH THE YEARS --> ROMANTASY GONE

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Set up paths to your data directories
train_dir = 'C:\Users\tuyen\DS4002P3\DATA\train_book_covers.csv'
val_dir = 'C:\Users\tuyen\DS4002P3\DATA\train_book_covers.csv'

# Define image dimensions and batch size
img_height, img_width = 224, 224
batch_size = 32

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Validation data should not have augmentation
val_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/train'

In [ ]:
# Load pre-trained ResNet50 with ImageNet weights, excluding the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze the ResNet50 layers

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)  # Output layer for genres

# Build the full model
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
epochs = 10  # Set the number of epochs

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)


In [ ]:
# Unfreeze specific layers for fine-tuning --> optional
for layer in base_model.layers[-10:]:  # Unfreeze the last 10 layers, for example
    layer.trainable = True

# Recompile with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
fine_tune_epochs = 5
history_fine = model.fit(
    train_generator,
    epochs=fine_tune_epochs,
    validation_data=val_generator
)


In [ ]:
test_loss, test_accuracy = model.evaluate(val_generator)
print(f"Test Accuracy: {test_accuracy:.2f}")
